<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from scipy import stats as st
import re
import nltk

**Задание 1**

**Ответьте на следующие вопросы:**

1) Как критики относятся к спортивным играм?
<br>2) Критикам нравятся больше игры на PC или на PS4?
<br>3) Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:

- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста. 

In [2]:
games = pd.read_csv('games.csv')
games.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


**1) Как критики относятся к спортивным играм?**

H0: средняя оценка спортивных игр критиками <= 70
<br>H1: средняя оценка спортивных игр критиками составляет более 70

Alpha здесь и далее = 0.05

**Сравниваем значение по выборке с константой, поэтому нужен одновыборочный критерий.**

In [156]:
alpha = 0.05
# Поскольку при nan_policy='omit' вылезала ошибка 
# "ValueError: nan-containing/masked inputs with nan_policy='omit' are currently not supported by one-sided alternatives."
# я пошел топорным путем: 
result = st.ttest_1samp(games.loc[games['Genre'] =='Sports']['Critic_Score'].dropna(), 70, alternative='greater')

print('Среднее для оценок спортивных игр =',games.loc[games['Genre'] =='Sports']['Critic_Score'].mean())
print('P-value =', result[1])

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, среднее больше 70 (спортивные игры критикам нравятся)')
else:
    print('Не отвергаем нулевую нулевую гипотезу, спортивные игры имеют невысокие оценки критиков')

Среднее для оценок спортивных игр = 72.03975799481418
P-value = 2.9598344399548874e-07
Отвергаем нулевую гипотезу, среднее больше 70 (спортивные игры критикам нравятся)


**2) Критикам нравятся больше игры на PC или на PS4?**


H0: средняя оценка критиков игр для PC и для PS4 одинаковая
<br>H1: средняя оценка критиков игр для PC и для PS4 различается

**Используем двухвыборочный двухсторонний критерий.**

Применяем критерий Уэлча

In [157]:
result = st.ttest_ind(games.loc[games['Platform'] =='PC']['Critic_Score'], games.loc[games['Platform'] =='PS4']['Critic_Score'], nan_policy='omit', equal_var=False)
print('P-Value =', result[1])

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка критиков игр для PC и для PS4 действительно различается')
    print('средняя оценка критиков игр на PC:', games.loc[games['Platform'] =='PC']['Critic_Score'].mean())
    print('медианное значение оценок критиков игр на PC:', games.loc[games['Platform'] =='PC']['Critic_Score'].median())
    print('средняя оценка критиков игр на PS4:', games.loc[games['Platform'] =='PS4']['Critic_Score'].mean())
    print('медианное значение оценок критиков игр на PS4:', games.loc[games['Platform'] =='PS4']['Critic_Score'].median())
else:
    print('Не отвергаем нулевую гипотезу. Cредняя оценка критиков игр для PC и для PS4 одинаковая, а значит критики одинаково относятся к играм на PС и PS4')


P-Value = 0.00016176478394495486
Отвергаем нулевую гипотезу, средняя оценка критиков игр для PC и для PS4 действительно различается
средняя оценка критиков игр на PC: 76.00302114803625
медианное значение оценок критиков игр на PC: 78.0
средняя оценка критиков игр на PS4: 72.43805309734513
медианное значение оценок критиков игр на PS4: 74.0


**3) Критикам больше нравятся стрелялки или стратегии?**

H0: средняя оценка критиков игр для стратегий и шутеров одинаковая 
<br>H1: средняя оценка критиков игр для стратегий и шутеров различается

**Используем двухвыборочный двухсторонний критерий.**

Применяем критерий Уэлча

В случае если верной оказывается альтернативная гипотеза, рассматриваем средние и медианные значения оценок шутеров и стратегий.

In [158]:
result = st.ttest_ind(games.loc[games['Genre'] =='Shooter']['Critic_Score'], games.loc[games['Genre'] =='Strategy']['Critic_Score'], nan_policy='omit', equal_var=False)
print('P-Value =', result[1])

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, средняя оценка Стратегий и Шутеров различается')
    print('средняя оценка стратегий критиками:', games.loc[games['Genre'] =='Strategy']['Critic_Score'].mean())
    print('медианное значение оценок стратегий критиками:', games.loc[games['Genre'] =='Strategy']['Critic_Score'].median())
    print('средняя оценка шутеров критиками:', games.loc[games['Genre'] =='Shooter']['Critic_Score'].mean())
    print('медианное значение оценок шутеров критиками:', games.loc[games['Genre'] =='Shooter']['Critic_Score'].median())
else:
    print('Не отвергаем нулевую гипотезу. Cредняя оценка критиков игр для PC и для PS4 одинаковая')

P-Value = 0.03371460711885051
Отвергаем нулевую гипотезу, средняя оценка Стратегий и Шутеров различается
средняя оценка стратегий критиками: 72.01748251748252
медианное значение оценок стратегий критиками: 73.0
средняя оценка шутеров критиками: 70.19117647058823
медианное значение оценок шутеров критиками: 73.0


**Задание 2**

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

1) Приведите весь текст к нижнему регистру;
<br>2) Удалите мусорные символы;
<br>3) Удалите стоп-слова;
<br>4) Приведите все слова к нормальной форме;
<br>5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:


    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df.Message)
    names = tfidf.get_feature_names()
    tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);

<br>6) Разделите данные на тестовые и тренировочные в соотношении **30/70**, укажите **random_state=42**. Используйте **train_test_split**;
<br>7) Постройте модель логистической регрессии, укажите **random_state=42**, оцените ее точность на тестовых данных;
<br>8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);
<br>9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [36]:
spam = pd.read_csv('spam.csv')
spam.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


**Олег, еще раз большое вам спасибо за помощь в Slack. По вашей рекомендации я переработал код, уйдя по максимуму от циклов**

**1) Приведите весь текст к нижнему регистру**

In [4]:
text = spam['Message'].str.lower()
text.head()

0    go until jurong point, crazy.. available only ...
1                        ok lar... joking wif u oni...
2    free entry in 2 a wkly comp to win fa cup fina...
3    u dun say so early hor... u c already then say...
4    nah i don't think he goes to usf, he lives aro...
Name: Message, dtype: object

**2) Удалите мусорные символы:**

In [5]:
text = text.str.replace('[\W_]+', ' ', regex = True)

text.head()

0    go until jurong point crazy available only in ...
1                             ok lar joking wif u oni 
2    free entry in 2 a wkly comp to win fa cup fina...
3         u dun say so early hor u c already then say 
4    nah i don t think he goes to usf he lives arou...
Name: Message, dtype: object

In [6]:
text = text.str.split(' ')
text.head()

0    [go, until, jurong, point, crazy, available, o...
1                     [ok, lar, joking, wif, u, oni, ]
2    [free, entry, in, 2, a, wkly, comp, to, win, f...
3    [u, dun, say, so, early, hor, u, c, already, t...
4    [nah, i, don, t, think, he, goes, to, usf, he,...
Name: Message, dtype: object

**3) Удалите стоп-слова:**

In [7]:
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

In [9]:
def removeStopWords(words):
    return [w for w in words if w not in stopwords_set] # ну, вот тут я ужеот цикла никуда не смог уйти
text = text.apply(removeStopWords)
text.head()

0    [go, jurong, point, crazy, available, bugis, n...
1                     [ok, lar, joking, wif, u, oni, ]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3      [u, dun, say, early, hor, u, c, already, say, ]
4       [nah, think, goes, usf, lives, around, though]
Name: Message, dtype: object

**4) Приведите все слова к нормальной форме**

In [10]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [11]:
def lemmatiser(words):
    return [wordnet_lemmatizer.lemmatize(w) for w in words] # аналогично, не смог понять, как тут без цикла можно реализовать
text = text.apply(lemmatiser)
text.head()

0    [go, jurong, point, crazy, available, bugis, n...
1                     [ok, lar, joking, wif, u, oni, ]
2    [free, entry, 2, wkly, comp, win, fa, cup, fin...
3      [u, dun, say, early, hor, u, c, already, say, ]
4          [nah, think, go, usf, life, around, though]
Name: Message, dtype: object

**5) Преобразуйте все сообщения в вектора TF-IDF:**

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(preprocessor=','.join) # (preprocessor=','.join) - позволяет уйти от ошибки 'list' object has no attribute 'lower'
tfidf_matrix = tfidf.fit_transform(text)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix.head()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix


In [13]:
X=tfidf_matrix
y=spam['Category']

**6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split:**

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

**7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных**

In [23]:
lr = LogisticRegression(random_state=42)

In [24]:
lr.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [25]:
y_pred = lr.predict(X_test)
print(y_pred)

['ham' 'ham' 'ham' ... 'ham' 'spam' 'ham']


In [33]:
accuracy_score(y_test, lr.predict(X_test))

0.958732057416268

**8) Опишите результаты при помощи confusion_matrix:**

In [27]:
confusion_matrix(y_test, y_pred)

array([[1445,    3],
       [  66,  158]], dtype=int64)

In [34]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp
print('Модель верно предсказала (верно отрицательный результат) НЕ СПАМ: %.0f раз' %(tn))
print('Модель НЕверно предсказала (ложно положительный результат) СПАМ: %.0f раза' %(fp))
print('Модель НЕверно предсказала (ложно отрицательный результат) НЕ СПАМ: %.0f раз' %(fn))
print('Модель верно предсказала (верно положительный результат) СПАМ: %.0f раз' %(tp))

Модель верно предсказала (верно отрицательный результат) НЕ СПАМ: 1445 раз
Модель НЕверно предсказала (ложно положительный результат) СПАМ: 3 раза
Модель НЕверно предсказала (ложно отрицательный результат) НЕ СПАМ: 66 раз
Модель верно предсказала (верно положительный результат) СПАМ: 158 раз


**9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного):**

Вот тут я, возможно, пошел каким-то излишне сложным путем (хотя, а как еще выделить конкретные 30%  DF, которые точно попали в X_test и y_test?), но это единственное, до чего я додумался...

In [29]:
X_origin=spam['Message']
y=spam['Category']

In [30]:
X_origin_train, X_origin_test, y_origin_train, y_origin_test = train_test_split(X_origin, y, test_size=0.30, random_state=42)

In [31]:
mistakes = pd.DataFrame()
# mistakes['Category'] = y_origin_test
mistakes['Messages'] = X_origin_test
mistakes['True Category'] = y_test
mistakes['Pred Category'] = y_pred

mistakes.head(150)

,Messages,True Category,Pred Category
3245,Squeeeeeze!! This is christmas hug.. If u lik ...,ham,ham
944,And also I've sorta blown him off a couple tim...,ham,ham
1044,Mmm thats better now i got a roast down me! i...,ham,ham
2484,Mm have some kanji dont eat anything heavy ok,ham,ham
812,So there's a ring that comes with the guys cos...,ham,ham
...,...,...,...
2650,You might want to pull out more just in case a...,ham,ham
4808,PRIVATE! Your 2004 Account Statement for 07849...,spam,spam
5022,:-( sad puppy noise,ham,ham
544,4 oclock at mine. Just to bash out a flat plan.,ham,ham


In [32]:
only_mistakes = mistakes.loc[mistakes['True Category'] != mistakes['Pred Category']]
only_mistakes

,Messages,True Category,Pred Category
2952,Hey now am free you can call me.,ham,spam
881,Reminder: You have not downloaded the content ...,spam,ham
1961,Guess what! Somebody you know secretly fancies...,spam,ham
3864,Oh my god! I've found your number again! I'm s...,spam,ham
2575,Your next amazing xxx PICSFREE1 video will be ...,spam,ham
...,...,...,...
4543,FreeMsg Hi baby wow just got a new cam moby. W...,spam,ham
752,You have an important customer service announc...,spam,ham
309,TheMob> Check out our newest selection of cont...,spam,ham
495,Are you free now?can i call now?,ham,spam
